In [1]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import os
import mra
from mra import *
import models
from models import *

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [6]:
def make_dataset(X, Y, f):
    x = tf.data.Dataset.from_tensor_slices(X)
    y = tf.data.Dataset.from_tensor_slices(Y)
    x = x.map(f)
    xy = tf.data.Dataset.zip((x, y))
    xy = xy.shuffle(10000)
    return xy

xy = np.loadtxt("../datasets/covtype.data",delimiter=',')
np.random.shuffle(xy)
x = xy[:,0:-1]
y = xy[:,-1]

x = (x - np.min(x, axis=0))/(np.max(x, axis=0) - np.min(x, axis=0))
y = y.reshape((len(x), 1)).astype("float32")-1
train_ds = make_dataset(x, y, lambda t: t)

In [12]:
main = {}

for split in [4]:
    print("split = {}".format(split))
    main[split] = mra(train_ds)
    make_f, make_g, make_generator = models.make_mlp(
        attr_num=54,
        class_num=7,
        split=split,
        fg_units=256,
        fg_act="relu",
        generator_units=[1024,512,256,128]
    )
    main[split].train(
        make_f,
        make_g,
        loss_fn=tf.keras.losses.SparseCategoricalCrossentropy(),
        batch_size=32,
        epoch=5,
        lr=0.001,
        verbose=True, log_every=30000
    )

split = 4
Iteration 30000: Training loss: 0.4542
Iteration 60000: Training loss: 0.2958
Iteration 90000: Training loss: 0.2474


In [13]:
log_attack = {}
for split in [4]:
    print("split level = {}".format(split))
    log_attack[split] = main[split].attack(
        attack_iter=100,
        make_generator=make_generator,
        input_noise="normal",
        input_z=True,
        model_leak=True,
        lr_x=0.0001,
        lr_f=0.00001,
        epoch=50,
        iter_x=40,
        iter_f=1,
        verbose=True, log_every=1
    )

split level = 4
Iteration 0001: RG-uniform: 0.3066, RG-normal: 0.2920, reconstruction validation: 0.1564
Iteration 0002: RG-uniform: 0.3007, RG-normal: 0.2847, reconstruction validation: 0.1816
Iteration 0003: RG-uniform: 0.3150, RG-normal: 0.2843, reconstruction validation: 0.1653
Iteration 0004: RG-uniform: 0.2971, RG-normal: 0.2928, reconstruction validation: 0.1723
Iteration 0005: RG-uniform: 0.3053, RG-normal: 0.2828, reconstruction validation: 0.1694
Iteration 0006: RG-uniform: 0.3071, RG-normal: 0.2779, reconstruction validation: 0.1566
Iteration 0007: RG-uniform: 0.2972, RG-normal: 0.2805, reconstruction validation: 0.1549
Iteration 0008: RG-uniform: 0.2990, RG-normal: 0.2799, reconstruction validation: 0.1540
Iteration 0009: RG-uniform: 0.3094, RG-normal: 0.2886, reconstruction validation: 0.1624
Iteration 0010: RG-uniform: 0.3088, RG-normal: 0.2858, reconstruction validation: 0.1490
Iteration 0011: RG-uniform: 0.3045, RG-normal: 0.2808, reconstruction validation: 0.1444
Itera

KeyboardInterrupt: 